#UE03 - SPARQL Query

Please complete the 10 tasks in the `2. SPARQL` sheet of `SemAI.jar` first, and then transfer the task descriptions and your solutiosn in executable form to this notebook.

## Preparation

Reuse imports and functions from https://github.com/jku-win-dke/SemAI/blob/main/V02_SPARQL.ipynb and load the solar system graph. Query the solar system graph to check that everything works fine. 

In [1]:
# Install required packages
!pip install -q rdflib

# Imports
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import FOAF , XSD , RDFS, NamespaceManager 

# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def sparql_construct(graph, query):
  result_graph = Graph(namespace_manager = g.namespace_manager)  # create a Graph object that reuses the namespace prefixes of the original graph
  result_graph += graph.query(query)                             # execute the construct query against the original graph and add the resulting graph to the new one
  return result_graph

def sparql_ask(graph, query):
  return bool(graph.query(query))      # an ASK query has a boolean result, which should be returned as such

# Load solar system graph
g = Graph()
g.parse("https://raw.githubusercontent.com/jku-win-dke/SemAI/main/data/solarsystem.ttl",format="turtle")

# Query solar system graph (to check that everything works fine)
df = sparql_select(g,"""
  SELECT ?planet ?apoapsis ?apoapsis_uom
  WHERE { 
    ?planet rdf:type dbo:Planet . 
    OPTIONAL { ?planet v:apoapsis [rdf:value ?apoapsis ; v:uom ?apoapsis_uom ].  }
  }
""")
df

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 2.1 MB/s eta 0:00:00


,planet,apoapsis,apoapsis_uom
0,:Mercury,0.467,unit:AU
1,:Venus,0.728,unit:AU
2,:Earth,1.017,unit:AU
3,:Earth,149597871,unit:KM
4,:Mars,1.666,unit:AU
5,:Jupiter,5.4588,unit:AU
6,:Saturn,9.0412,unit:AU
7,:Uranus,20.11,unit:AU
8,:Neptune,30.33,unit:AU



## Task 1 (1 pt)

Geben Sie alle Zwergplaneten (Instanzen der Klasse dbo:DwarfPlanet) aus und falls vorhanden deren genaue Entsprechung (skos:exactMatch). Ordnen Sie das Ergebnis aufsteigend nach den URIs der Zwergplaneten.

In [2]:
query = """
  SELECT ?d ?match
  WHERE { 
    ?d a dbo:DwarfPlanet .
    OPTIONAL {?d skos:exactMatch ?match}
  }
"""

df = sparql_select(g, query, use_prefixes=False)
df

,d,match
0,http://dke.jku.at/example/solarsystem/Ceres,http://dbpedia.org/resource/1_Ceres
1,http://dke.jku.at/example/solarsystem/Pluto,http://dbpedia.org/resource/Pluto
2,http://dke.jku.at/example/solarsystem/Haumea,None
3,http://dke.jku.at/example/solarsystem/Makemake,None
4,http://dke.jku.at/example/solarsystem/Eris,None


## Task 2 (1 pt)

Ermitteln Sie alle Sterne, ihr exactMatch, und ihre Masse (geben Sie den Wert und die Maßeinheit aus). Ordnen Sie das Ergebnis nach der URI der Sterne

In [3]:
query = """
  SELECT ?star ?match ?massVal ?massUoM
  WHERE { 
    ?star skos:exactMatch ?match;
      v:mass ?mass;
      rdf:type dbo:Star .

    ?mass rdf:value ?massVal;
      v:uom ?massUoM . 
  }
  ORDER BY ?star

"""

df = sparql_select(g, query, use_prefixes=False)
df

,star,match,massVal,massUoM
0,http://dke.jku.at/example/solarsystem/AlphaCen...,http://www.wikidata.org/entity/Q2090157,1.1,http://dke.jku.at/example/examplevocabulary/So...
1,http://dke.jku.at/example/solarsystem/AlphaCen...,http://www.wikidata.org/entity/Q1052548,0.9,http://dke.jku.at/example/examplevocabulary/So...
2,http://dke.jku.at/example/solarsystem/ProximaC...,http://www.wikidata.org/entity/Q14266,0.1221,http://dke.jku.at/example/examplevocabulary/So...
3,http://dke.jku.at/example/solarsystem/Sun,http://dbpedia.org/resource/Sun,1.9884e+30,http://qudt.org/vocab/unit/KG


## Task 3 (1 pt)
Die Planeten unseres Sonnensystems und ihre jeweilige Anzahl an Monden. Unterscheiden Sie die im RDF-Graph beschriebenen Monde und die im RDF-Graph erfasste Anzahl von Monden. Sortieren Sie nach den Planeten.

In [4]:
query = """
SELECT ?planet ?assertedNo (COUNT(?moon) AS ?noOfDescribedMoons)
WHERE { 
  ?planet a dbo:Planet;
          v:orbits :Sun .
	OPTIONAL { ?planet v:nrOfMoons ?assertedNo . }
	OPTIONAL { 
    ?moon a dbo:Satellite;
          v:orbits ?planet .
  }
}
GROUP BY ?planet ?assertedNo
ORDER BY ?planet

"""

df = sparql_select(g, query, use_prefixes=False)
df

,planet,assertedNo,noOfDescribedMoons
0,http://dke.jku.at/example/solarsystem/Earth,1,1
1,http://dke.jku.at/example/solarsystem/Jupiter,79,4
2,http://dke.jku.at/example/solarsystem/Mars,2,2
3,http://dke.jku.at/example/solarsystem/Mercury,0,0
4,http://dke.jku.at/example/solarsystem/Neptune,14,0
5,http://dke.jku.at/example/solarsystem/Saturn,82,2
6,http://dke.jku.at/example/solarsystem/Uranus,27,0
7,http://dke.jku.at/example/solarsystem/Venus,0,0


## Task 4 (1 pt)
Ermitteln sie für die Planeten in unserem Sonnensystem die durchschnittliche Anzahl an Monden (die auch im RDF-Graph beschrieben sind) pro Planet.

In [5]:
query = """
SELECT (AVG(?noOfDescribedMoons) AS ?avgNoOfDescribedMoons) 
WHERE {
        {
            SELECT ?planet (COUNT(?moon) AS ?noOfDescribedMoons)
            WHERE { 
              ?planet a dbo:Planet;
                      v:orbits :Sun .
              OPTIONAL { 
                ?moon a dbo:Satellite;
                      v:orbits ?planet .
              }
            }
            GROUP BY ?planet
        }
}

"""

df = sparql_select(g, query, use_prefixes=False)
df

,avgNoOfDescribedMoons
0,1.125


## Task 5 (1 pt)
Geben Sie die im RDF-Graph verwendeten Klassen und ihre Anzahl an Instanzen aus. Geben Sie nur Klassen mit mindestens 2 Instanzen aus. Ordnen Sie die Ausgabe nach der URI der Klassen.

In [6]:
query = """
SELECT ?class (COUNT(*) AS ?noOfInstances)
WHERE { 
  ?s a ?class .
}
GROUP BY ?class
HAVING (COUNT(*) > 2)
ORDER BY ?class

"""

df = sparql_select(g, query, use_prefixes=False)
df



,class,noOfInstances
0,http://dbpedia.org/ontology/DwarfPlanet,5
1,http://dbpedia.org/ontology/Planet,8
2,http://dbpedia.org/ontology/Satellite,9
3,http://dbpedia.org/ontology/Star,4


## Task 6 (1 pt)
Geben Sie die im RDF-Graph verwendeten Properties und ihre Häufigkeit je Klasse aus. Gezählt werden sollen die Instanzen der Klasse, die die Property ausprägen. Geben Sie nur Properties zu Klassen aus, die von mindestens 7 Instanzen dieser Klasse ausgeprägt werden. Ordnen Sie die Ausgabe zuerst nach der Klasse und dann nach der Property.

In [7]:
query = """
SELECT ?class ?prop (COUNT(DISTINCT ?instance) as ?noOfInstances)
WHERE { 
  ?instance a ?class .
  ?instance ?prop ?o .
}
GROUP BY ?class ?prop
HAVING (COUNT(DISTINCT ?instance) >= 7)
ORDER BY ?class ?prop

"""

df = sparql_select(g, query, use_prefixes=False)
df

,class,prop,noOfInstances
0,http://dbpedia.org/ontology/Planet,http://dke.jku.at/example/examplevocabulary/ap...,8
1,http://dbpedia.org/ontology/Planet,http://dke.jku.at/example/examplevocabulary/nr...,8
2,http://dbpedia.org/ontology/Planet,http://dke.jku.at/example/examplevocabulary/or...,8
3,http://dbpedia.org/ontology/Planet,http://dke.jku.at/example/examplevocabulary/ra...,7
4,http://dbpedia.org/ontology/Planet,http://dke.jku.at/example/examplevocabulary/te...,7
5,http://dbpedia.org/ontology/Planet,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,8
6,http://dbpedia.org/ontology/Planet,http://www.w3.org/2004/02/skos/core#exactMatch,8
7,http://dbpedia.org/ontology/Satellite,http://dke.jku.at/example/examplevocabulary/or...,9
8,http://dbpedia.org/ontology/Satellite,http://dke.jku.at/example/examplevocabulary/ra...,9
9,http://dbpedia.org/ontology/Satellite,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,9


## Task 7 (1 pt)
Ermitteln Sie alle Sterne, sowie alle Monde. Zu jedem Mond ermitteln Sie auch den Planet, den dieser Mond umrundet. Ordnen Sie die Ausgabe nach der URI der Sterne und Monde.

In [8]:
query = """
SELECT *
WHERE { 
  { 
    ?x a dbo:Star .
  }
  UNION
  { 
    ?x a dbo:Satellite;
          v:orbits ?y .
  } 
}
ORDER BY ?x
"""

df = sparql_select(g, query, use_prefixes=False)
df

,y,x
0,None,http://dke.jku.at/example/solarsystem/AlphaCen...
1,None,http://dke.jku.at/example/solarsystem/AlphaCen...
2,http://dke.jku.at/example/solarsystem/Jupiter,http://dke.jku.at/example/solarsystem/Callisto
3,http://dke.jku.at/example/solarsystem/Mars,http://dke.jku.at/example/solarsystem/Deimos
4,http://dke.jku.at/example/solarsystem/Saturn,http://dke.jku.at/example/solarsystem/Enceladus
5,http://dke.jku.at/example/solarsystem/Jupiter,http://dke.jku.at/example/solarsystem/Europa
6,http://dke.jku.at/example/solarsystem/Jupiter,http://dke.jku.at/example/solarsystem/Ganymede
7,http://dke.jku.at/example/solarsystem/Jupiter,http://dke.jku.at/example/solarsystem/Io
8,http://dke.jku.at/example/solarsystem/Saturn,http://dke.jku.at/example/solarsystem/Mimas
9,http://dke.jku.at/example/solarsystem/Earth,http://dke.jku.at/example/solarsystem/Moon


[Linktext](https://)## Task 8 (1 pt)
Erzeugen sie mittels einer Construct-Query den angezeigten RDF-GRAPH (Expected Result). Reihenfolge ist unerheblich.

In [9]:
query = """
CONSTRUCT{
  :Sun v:wirdUmrundetVon ?sunOrbiter .

  ?sunOrbiter a v:Himmelskoerper ;
    v:wirdUmrundetVon ?planetOrbiter .

  ?planetOrbiter a v:Himmelskoerper .
}
WHERE { 
  ?sunOrbiter v:orbits :Sun;
              a dbo:Planet .

  OPTIONAL{
    ?planetOrbiter v:orbits ?sunOrbiter;
                      a dbo:Satellite .
  }
}
"""

g2 = sparql_construct(g, query)
print(g2.serialize(format="turtle"))

@prefix : <http://dke.jku.at/example/solarsystem/> .
@prefix v: <http://dke.jku.at/example/examplevocabulary/> .

:Sun v:wirdUmrundetVon :Earth,
        :Jupiter,
        :Mars,
        :Mercury,
        :Neptune,
        :Saturn,
        :Uranus,
        :Venus .

:Callisto a v:Himmelskoerper .

:Deimos a v:Himmelskoerper .

:Earth a v:Himmelskoerper ;
    v:wirdUmrundetVon :Moon .

:Enceladus a v:Himmelskoerper .

:Europa a v:Himmelskoerper .

:Ganymede a v:Himmelskoerper .

:Io a v:Himmelskoerper .

:Jupiter a v:Himmelskoerper ;
    v:wirdUmrundetVon :Callisto,
        :Europa,
        :Ganymede,
        :Io .

:Mars a v:Himmelskoerper ;
    v:wirdUmrundetVon :Deimos,
        :Phobos .

:Mercury a v:Himmelskoerper .

:Mimas a v:Himmelskoerper .

:Moon a v:Himmelskoerper .

:Neptune a v:Himmelskoerper .

:Phobos a v:Himmelskoerper .

:Saturn a v:Himmelskoerper ;
    v:wirdUmrundetVon :Enceladus,
        :Mimas .

:Uranus a v:Himmelskoerper .

:Venus a v:Himmelskoerper .




## Task 9 (1 pt)
Gibt es einen Stern im RDF-Graph, der massereicher als die Sonne ist.

In [11]:
query = """
ASK
{ 
  :Sun v:mass ?sunMass .

  ?sunMass rdf:value ?sunMassValue .

  ?otherStar a dbo:Star;
      v:mass ?otherMass .

  ?otherMass rdf:value ?otherMassValue ;
            v:uom ?otherMassUOM .

    FILTER
    (
      (?otherMassUOM = v:SolarMass && ?otherMassValue > 1 )
      ||
      (?otherMassUOM = v:KG && ?otherMassValue > ?sunMassValue)
    )
}
"""

a = sparql_ask(g, query)
print(a)

True


## Task 10 (1 pt)
Geben Sie eine Beschreibung der Resourcen aus, die einen Durchmesser zwischen 20.000 km und 30.000 km haben.



In [ ]:

query = """
DESCRIBE ?resource
WHERE { 
  ?resource v:radius ?radius .

  ?radius rdf:value ?radiusValue ;
          v:uom unit:KM .

  FILTER
  (?radiusValue >= 20000 && ?radiusValue <= 30000)
}

"""

g2 = sparql_construct(g, query)
print(g2.serialize(format="turtle"))